## nc 파일 데이터 확인

In [43]:
# pip install xarray

import xarray as xr

file_path = '/home/yurim2/WRF/WRF-4.1.2/test/em_real/wrfout_d01_2024-01-01_04:00:00'
ds = xr.open_dataset(file_path)

print(ds)

<xarray.Dataset>
Dimensions:                (Time: 1, south_north: 179, west_east: 179,
                            bottom_top: 32, bottom_top_stag: 33,
                            soil_layers_stag: 4, west_east_stag: 180,
                            south_north_stag: 180, seed_dim_stag: 8)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 ...
    XLONG                  (Time, south_north, west_east) float32 ...
    XTIME                  (Time) datetime64[ns] ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 ...
    XLONG_U                (Time, south_north, west_east_stag) float32 ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 ...
    XLONG_V                (Time, south_north_stag, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_s

In [56]:
xlong = ds['XLONG'].isel(Time = 0)

print(xlong)

<xarray.DataArray 'XLONG' (south_north: 179, west_east: 179)>
[32041 values with dtype=float32]
Coordinates:
    XLAT     (south_north, west_east) float32 ...
    XLONG    (south_north, west_east) float32 ...
    XTIME    datetime64[ns] ...
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LONGITUDE, WEST IS NEGATIVE
    units:        degree_east
    stagger:      


In [45]:
xlong.values

array([[107.773865, 108.0043  , 108.23511 , ..., 150.25964 , 150.48965 ,
        150.7193  ],
       [107.71118 , 107.94223 , 108.173645, ..., 150.32242 , 150.55304 ,
        150.78333 ],
       [107.64813 , 107.87979 , 108.11182 , ..., 150.38556 , 150.61682 ,
        150.84769 ],
       ...,
       [ 86.047485,  86.4487  ,  86.852295, ..., 172.01074 , 172.40924 ,
        172.80536 ],
       [ 85.80841 ,  86.21094 ,  86.615906, ..., 172.25    , 172.64978 ,
        173.04712 ],
       [ 85.566956,  85.970795,  86.377075, ..., 172.49158 , 172.89264 ,
        173.29126 ]], dtype=float32)

In [50]:
print('u dim : ', ds['U'].dims)

u dim :  ('Time', 'bottom_top', 'south_north', 'west_east_stag')


In [ ]:
def get_mean_dim(ds, var_name, time_index=0):
    """
    주어진 변수의 적절한 차원을 찾아 평균을 계산하여 반환.
    
    Parameters:
    - ds: xarray.Dataset - 데이터셋
    - var_name: str - 변수 이름 (예: 'U' 또는 'V')
    - time_index: int - 시간 인덱스 (기본값은 0)
    
    Returns:
    - xarray.DataArray - 평균이 적용된 변수
    """
    variable = ds[var_name]
    possible_dims = ['bottom_top', 'bottom_top_stag'] 

    for dim in possible_dims:
        if dim in variable.dims:
            return variable.mean(dim=dim).isel(Time=time_index)
    
    raise ValueError(f"{var_name} 변수에서 사용 가능한 차원을 찾을 수 없습니다.")

u = get_mean_dim(ds, 'U', time_index = 0)
print('u_dim : ', u.values)


u_dim :  [[-3.3352785 -3.5699372 -3.5892205 ... -6.308424  -6.3838296 -6.4644513]
 [-2.2302995 -2.3912618 -2.4206657 ... -6.24735   -6.3383875 -6.440873 ]
 [-0.9662071 -0.990472  -1.0034723 ... -6.1830664 -6.282534  -6.3996987]
 ...
 [20.067257  20.166946  20.274221  ... 11.485388  11.201811  11.09444  ]
 [19.37067   19.478798  19.6346    ... 11.6575    11.343041  11.199776 ]
 [18.680733  18.743292  18.810713  ... 11.660259  11.432852  11.3228035]]


## 데이터 db에 적재

In [59]:
import os
import psycopg2
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle

In [42]:
def get_db_connection():
    try:
        conn = psycopg2.connect(
            host="172.27.80.1",
            database="calpuff",
            user="postgres",
            password="1201"
        )
        print("데이터베이스에 연결되었습니다.")
        return conn
    except Exception as e:
        print("데이터베이스 연결에 실패했습니다:", e)
        return None

In [57]:
def process_nc_file(file_path):
    files = os.path.basename(file_path)
    parts = files.split('_')
    if len(parts) >= 4:
        timestamp_str = f"{parts[2]} {parts[3].replace(':', '-')}"
        timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %H-%M-%S')
    else:
        print(f'{file_path}의 형식이 잘못되었습니다.')
        return False  # 실패했음을 표시하기 위해 False 반환

    ds = xr.open_dataset(file_path)
    
    # 임시 파일 경로 형성 (f-string 형식 사용)
    temp_file = f"/tmp/{timestamp}.nc"
    ds.to_netcdf(temp_file)
    ds.close()

    # 임시 파일을 바이너리로 읽기
    with open(temp_file, 'rb') as f:
        binary_data = f.read()
    
    # 데이터베이스 연결
    conn = get_db_connection()
    if conn is None:
        return False
    
    cursor = conn.cursor()
    
    try:
        # 데이터베이스에 바이너리 데이터 삽입
        cursor.execute(
            "INSERT INTO WRF_2024_01_NC (timestamp, nc_data) VALUES (%s, %s)",
            (timestamp, psycopg2.Binary(binary_data))
        )
        
        conn.commit()
        print(f'{timestamp} 데이터가 성공적으로 삽입되었습니다.')
    except Exception as e:
        print(f'{timestamp} 데이터 삽입에 실패했습니다:', e)
        return False
    finally:
        cursor.close()
        conn.close()
        # 임시 파일 삭제
        os.remove(temp_file)
    
    return True  # 성공했음을 표시하기 위해 True 반환

def process_all_file(folder_path):
    success_count = 0
    fail_count = 0
    
    # 폴더에서 파일 찾기
    files = [file_name for file_name in os.listdir(folder_path) if file_name.startswith('wrfout_d01_2024-01-01')]
    files = sorted(files)
    
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        if process_nc_file(file_path):
            success_count += 1
        else:
            fail_count += 1

    print(f'\n처리 완료:')
    print(f'성공: {success_count} 파일')
    print(f'실패: {fail_count} 파일')
    
def main():
    print('WRF 데이터 nc 파일 변환 시작 ...')
    folder_path = '/home/yurim2/WRF/SQL'
    process_all_file(folder_path)
    print('변환 작업 완료')
    
if __name__ == '__main__':
    main()


WRF 데이터 nc 파일 변환 시작 ...
데이터베이스에 연결되었습니다.
2024-01-01 01:00:00 데이터가 성공적으로 삽입되었습니다.
데이터베이스에 연결되었습니다.
2024-01-01 02:00:00 데이터가 성공적으로 삽입되었습니다.
데이터베이스에 연결되었습니다.
2024-01-01 03:00:00 데이터가 성공적으로 삽입되었습니다.
데이터베이스에 연결되었습니다.
2024-01-01 04:00:00 데이터가 성공적으로 삽입되었습니다.

처리 완료:
성공: 4 파일
실패: 0 파일
변환 작업 완료
